#### Monday, November 4, 2024

mamba activate langchain

[Build an Agent](https://python.langchain.com/docs/tutorials/agents/)

The sample code uses Anthropic. To use it requires a paid plan, which is not by tokens but a monthly cost. So yeah, I am not gonna sign up for Anthropic because I think I will rarely use it, so I am going to switch to OpenAI in this code. 

Locally, I am using 'hermes-3-llama-3.1-8b' from LMStudio.

This all runs without errors in one pass.


In [ ]:
import os

print(os.environ["LANGCHAIN_TRACING_V2"])
print(os.environ["LANGCHAIN_API_KEY"])
print(os.environ["OPENAI_API_KEY"])
print(os.environ["TAVILY_API_KEY"])
print(os.environ["ANTHROPIC_API_KEY"])

#### End-to-end agent

In [2]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [3]:
# Create the agent
memory = MemorySaver()

# just say no to Anthropic kids ... 
# model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
from langchain_openai import ChatOpenAI
# model = ChatOpenAI(model="gpt-3.5-turbo")

model = ChatOpenAI(base_url="http://localhost:1234/v1", 
                   model = "hermes-3-llama-3.1-8b",  # do not pass in an unrecognized model name ... 
                   api_key="lm-studio", 
                   temperature=0)


search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [4]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! San Francisco is a great city to live in. What do you like most about living there? Are there any favorite spots or activities you enjoy in the city?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 18, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-5c6e537e-b029-47b8-9eb6-677f9b787060-0', usage_metadata={'input_tokens': 18, 'output_tokens': 35, 'total_tokens': 53})]}}
----
{'agent': {'messages': [AIMessage(content="I don't have real-time access to current weather data, but I can provide some general information for San Francisco:\n\nSan Francisco has a Mediterranean climate with cool, wet winters and mild, dry summers. On average, January is the coldest month with highs around 58°F (14°C) an

#### Define Tools

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1730742258, 'localtime': '2024-11-04 09:44'}, 'current': {'last_updated_epoch': 1730741400, 'last_updated': '2024-11-04 09:30', 'temp_c': 13.2, 'temp_f': 55.7, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.7, 'wind_kph': 4.3, 'wind_degree': 17, 'wind_dir': 'NNE', 'pressure_mb': 1023.0, 'pressure_in': 30.21, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 58, 'cloud': 0, 'feelslike_c': 13.0, 'feelslike_f': 55.4, 'windchill_c': 13.0, 'windchill_f': 55.4, 'heatindex_c': 13.2, 'heatindex_f': 55.7, 'dewpoint_c': 4.9, 'dewpoint_f': 40.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 4.5, 'gust_kph': 7.2}}"}, {'url': 'https://world-weather.info/forecast/usa/san_fra

#### Using Language Models

In [6]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-4")
# model = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hello! How can I assist you today? Feel free to ask me anything and I'll do my best to help or provide information on a wide range of topics."

In [8]:
model_with_tools = model.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today? Feel free to ask me anything and I'll do my best to help or provide information on a wide range of topics.
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: The weather in San Francisco, California can vary greatly depending on the time of year. Generally, San Francisco experiences cool summers and mild winters.

As of my last update in April 2023, without a specific date or time provided, I can't give you an exact current weather report. However, here's a general idea:

- **Summer (June to August)**: Temperatures typically range from the mid-50s to low 70s °F (around 10-21°C). It's usually cooler and foggy near the coast, with warmer temperatures in inland areas.

- **Winter (December to February)**: Temperatures can range from the upper 40s to low 60s °F (around 9-15°C). Fog is common, especially during the night and early morning hours.

Keep in mind that San Francisco's weather can be quite changeable, with microclimates across the city due to its unique geography. It's always a good idea to check a reliable weather forecast service for the most accurate and up-to-date information before planning your day.
ToolCalls: []


#### Create the Agent

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

#### Run the Agent

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='fb42af81-d2ec-425b-b828-b9cc9a16de1e'),
 AIMessage(content="Hello! How can I assist you today? Feel free to ask me anything and I'll do my best to help or provide information on a wide range of topics.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 11, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-9a99f01b-1246-4d9b-a23c-bcc516592573-0', usage_metadata={'input_tokens': 11, 'output_tokens': 33, 'total_tokens': 44})]

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='6e239761-ce15-4f61-bd10-769cc2ece121'),
 AIMessage(content="To provide you with an accurate and up-to-date weather forecast for San Francisco, I would need to access a reliable weather service or API. However, I can give you a general idea of what to expect based on current trends.\n\nAs of March 2023, San Francisco's weather is typically mild and cool, especially considering its coastal location in Northern California. The city experiences a Mediterranean climate with warm, dry summers and cool, wet winters.\n\nIn the summer months (June-August), temperatures usually range from the mid-50s to low 70s Fahrenheit (around 12-21°C). However, heatwaves can occasionally push temperatures into the high 80s F (around 29°C) or above.\n\nDuring winter (December-February), temperatures are cooler and more variable, ranging from the low 40s to mid-60s F (around 4-18°C). Rain is common during this season, with an average of about 3-5 inches (76

#### Streaming Messages

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="To provide you with an accurate and up-to-date weather forecast for San Francisco, I would need to access a reliable weather service or API. However, I can give you a general idea of what to expect based on typical weather patterns in San Francisco.\n\nSan Francisco is known for its mild climate, but it can be quite variable depending on the time of year:\n\n1. Summer (June-August): Temperatures usually range from the mid-50s to low 70s Fahrenheit (around 10-21°C). It's generally warm and dry, with occasional fog in the mornings or evenings.\n\n2. Fall (September-November): Temperatures start to cool down, ranging from the mid-50s to low 60s Fahrenheit (around 10-15°C). Fog becomes more common, especially in the mornings.\n\n3. Winter (December-February): Temperatures are generally cool and damp, with averages ranging from the mid-40s to low 60s Fahrenheit (around 7-16°C). Rain is more frequent during this season, but snow is extremely rare.\n

#### Streaming Tokens

In [15]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

To| provide| you| with| an| accurate| and| up|-to|-date|

/tmp/ipykernel_32886/2442860291.py:1: LangChainBetaWarning: This API is in beta and may change in the future.
  async for event in agent_executor.astream_events(


 weather| forecast| for| San| Francisco|,| I| would| need| to| access| a| reliable| weather| service| or| API|.| However|,| I| can| give| you| a| general| idea| of| what| to| expect| based| on| typical| weather| patterns| in| San| Francisco|.

|San| Francisco| is| known| for| its| mild| climate|,| but| it| can| be| quite| variable| depending| on| the| time| of| year|:

|1|.| Summer| (|June|-Aug|ust|):| Temper|atures| usually| range| from| the| mid|-|50|s| to| low| |70|s| Fahrenheit| (|around| |10|-|21|°C|).| It|'s| generally| warm| and| dry|,| with| occasional| fog| in| the| mornings| or| evenings|.

|2|.| Fall| (|September|-Nov|ember|):| Temper|atures| start| to| cool| down|,| ranging| from| the| mid|-|50|s| to| low| |60|s| Fahrenheit| (|around| |10|-|15|°C|).| Fog| becomes| more| common|,| especially| in| the| mornings|.

|3|.| Winter| (|December|-|February|):| Temper|atures| are| generally| cool| and| damp|,| with| averages| ranging| from| the| mid|-|40|s| to| low| |60|s| Fahrenheit

#### Adding in memory

In [16]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask and I'll do my best to help you out.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 13, 'total_tokens': 58, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e480a9c-2dd7-4ad7-aa9d-e6991c206399-0', usage_metadata={'input_tokens': 13, 'output_tokens': 45, 'total_tokens': 58})]}}
----


In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob, as you mentioned in your initial greeting. Is there anything else I can help you with today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 73, 'total_tokens': 99, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-59ffe902-e1c0-4748-8d9e-f0f348a23fdd-0', usage_metadata={'input_tokens': 73, 'output_tokens': 26, 'total_tokens': 99})]}}
----


In [20]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I do not have enough context to determine your name. If you would like me to call you by a specific name, please let me know and I will be happy to use it when referring to you in our conversation. Otherwise, you can simply provide me with your name whenever you'd like.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 14, 'total_tokens': 73, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-2102c44e-bccb-41c2-b027-94824b12692c-0', usage_metadata={'input_tokens': 14, 'output_tokens': 59, 'total_tokens': 73})]}}
----
